# Accelerating metagenomic analysis with [Graphistry](graphistry.com) focusing on viral tracing over time

## [viral calling pipeline here](https://github.com/dcolinmorgan/viral_snake)

Using GPU-accelerated UMAP + DBScan analysis & visualization, metagenomic samples' bacterial compositions can be clustered and compared faster and much more easily explored.

*   Task: Analyze metagenomic samples for similarity
*   Data: time series samples
*   563 samples collected from 84 donors, producing 4 dense long-term time series (up to 1 sample every other day during 18 months)
* Clustering: the species component extracted from time-stamped patient samples, e.g., tuple of <idx=0, time=1, donor=aa, species=abc>
* Each **node** is <idx=0, rel_time=0, value=abc>, and clustering is on species=abc text similarity
* *n.b.* since species text is full taxa information, text comparison can return degree similarity

*   [data](https://www.ebi.ac.uk/ena/browser/view/PRJNA544527)
*   [metadata](https://static-content.springer.com/esm/art%3A10.1038%2Fs41591-019-0559-3/MediaObjects/41591_2019_559_MOESM3_ESM.xlsx)
*   [paper](https://sci-hub.se/10.1038/s41591-019-0559-3)


**Insight/ Result:**

* 43s to umap and dbscan vs 2342s on a small T4 GPU
* over **50X** faster for a single run
* since [the reference paper for this analysis](https://journals.asm.org/doi/full/10.1128/msystems.00118-23) runs this analysis 12x per dataset (here we only have 1 dataset), we could expect to save nearly the entire 8hrs for this dataset, taking less than 10 minutes in total

# Setup

In [ ]:
!pip install -q --extra-index-url=https://pypi.nvidia.com cuml-cu12
import cuml,cudf
print(cuml.__version__)

!pip -q install graphistry[ai]

!pip install -q Biopython

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# import /configure

visualization step, get a free API key at https://hub.graphistry.com


In [ ]:
import numpy as np
import pandas as pd
import graphistry
from time import time

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key
graphistry.__version__

import cuml,cudf
print(cuml.__version__)

24.06.01


# bio-ml dataset


1.   [3 subjects x 10 time points](
https://www.ebi.ac.uk/ena/browser/view/PRJNA544527)

2.  [metadata](
https://static-content.springer.com/esm/art%3A10.1038%2Fs41591-019-0559-3/MediaObjects/41591_2019_559_MOESM3_ESM.xlsx)

In [ ]:
!wget https://gist.githubusercontent.com/lmeyerov/61a6a7d5fa0dbe51e786ed52408ac360/raw/11a11aa0b865ceb96880b2cd2ae3b12f1ef947c8/gistfile1.txt -O PRJNA544527_mpa4out.txt

In [ ]:
%%bash
if [ ! -f PRJNA544527_mpa4out.txt ]; then
    !wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR922/006/SRR9224006/SRR9224006_1.fastq.gz
    !wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR922/006/SRR9224006/SRR9224006_2.fastq.gz

    !gunzip SRR9224006_1.fastq.gz
    !gunzip SRR9224006_2.fastq.gz

    !head /content/SRR9224006_1.fastq
fi

In [ ]:
import os
if not os.path.exists('PRJNA544527_mpa4out.txt'):
    from Bio import SeqIO
    import glob,os
    import pandas as pd
    B=pd.DataFrame()
    for i in glob.glob('/content/*.fastq'):
        # j=os.path.basename(i)
        fasta_sequences = SeqIO.parse(open(i),'fastq')
        identifiers = []
        sequences = []
        for fasta in fasta_sequences:
            name, sequence = fasta.id, str(fasta.seq)
            identifiers.append(name)
            sequences.append(sequence)

        A=pd.DataFrame([identifiers,sequences]).T
        A.columns=['ID','seq']
        A.dropna(inplace=True)
        B=B.append(A)
        # A['ID']#=A.ID.str.split('-')[0:1]
    # B['ID']=B['ID'].str.split('-').str[0]+'_'+B['ID'].str.split('-').str[1]#.cat()
    B['ID']=B.ID.str.split('_length').str[0]
    B.index=B.ID

# install [HUMAnN 3](https://huttenhower.sph.harvard.edu/humann), a method for efficiently and accurately profiling the abundance of microbial metabolic pathways and other molecular functions from metagenomic or metatranscriptomic sequencing data.

### takes very long for running all samples
 (1day+ run on cluster)

In [ ]:
%%bash

if [ ! -f PRJNA544527_mpa4out.txt ]; then

    pip install humann --no-binary :all:
    pip install metaphlan

    humann_databases --download utility_mapping full /path/to/databases --update-config yes

    # humann_test
    wget https://github.com/biobakery/humann/raw/master/examples/demo.fastq.gz
    humann -i demo.fastq.gz -o sample_results


    mkdir assemble epi_sam_out mpa4_out
    humann -i /content/All_MAGs/Sample_101_S75_bin_1.fa -o test_out


    seq=$(ls /content/*.fastq | cut -d / -f2| cut -d _ -f1)

    for i in $(eval "echo "$seq" | cut -d _ -f1")

    do
    metaphlan /content/${i}.fa --nproc 40 --input_type fasta -o /content/assemble/${i}/h4_out.txt -t rel_ab_w_read_stats
    done
fi

# umap and dbscan

idea for metagenomic analysis based on [Quantifying Shared and Unique Gene Content across 17 Microbial Ecosystems
](https://journals.asm.org/doi/full/10.1128/msystems.00118-23)

(analyze all samples run on cluster)

also this [paper](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0997-x?ref=https://codemonkey.link#Sec7) and [method](https://github.com/marbl/Mash/blob/master/INSTALL.txt)



In [ ]:
data=pd.read_csv('/content/PRJNA544527_mpa4out.txt',sep='\t',skiprows=1,index_col=0)
data.index=data.reset_index().clade_name.str.split('|',expand=True)[6]
data=data.reset_index().dropna(axis=0)
data.index=data[6]
data=data.drop(columns=6)

!wget https://gist.githubusercontent.com/lmeyerov/b650f1ef9e56c3f1888ebb009bc5ed46/raw/76dda5fabcdfbcdf0cc58450982fbeb4b2e38a98/PRJNA544527-meta_inf.txt
meta=pd.read_csv('/content/PRJNA544527-meta_inf.txt',sep='\t',header=None)

mm=pd.merge(data.T,meta[[3,5]],left_index=True,right_on=3)

mm['id']=mm[5].str.split('-').str[0]
mm['time']=mm[5].str.split('_').str[0].str.split('-').str[1]

!wget https://static-content.springer.com/esm/art%3A10.1038%2Fs41591-019-0559-3/MediaObjects/41591_2019_559_MOESM3_ESM.xlsx
metaa=pd.read_excel('/content/41591_2019_559_MOESM3_ESM.xlsx',sheet_name='SupTable2',skiprows=3)
metaa=metaa[['Donor','Age','Sex','BMI']]

Full_table=pd.merge(mm,metaa,left_on='id',right_on='Donor')
Full_table=Full_table.drop(columns=[3,	5,	'id'])

data2=Full_table.melt(id_vars=['time','Donor','Age','Sex','BMI'])

data2=data2.rename(columns={'variable':'species'})
data2=data2.sort_values(by=['Donor','time','value'])

--2024-07-09 07:23:29--  https://raw.githubusercontent.com/dcolinmorgan/grph/main/PRJNA544527-meta_inf.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45603 (45K) [text/plain]
Saving to: ‘PRJNA544527-meta_inf.txt’

PRJNA544527-meta_in 100%[===================>]  44.53K  --.-KB/s    in 0.007s  

2024-07-09 07:23:29 (6.40 MB/s) - ‘PRJNA544527-meta_inf.txt’ saved [45603/45603]

--2024-07-09 07:23:29--  https://static-content.springer.com/esm/art%3A10.1038%2Fs41591-019-0559-3/MediaObjects/41591_2019_559_MOESM3_ESM.xlsx
Resolving static-content.springer.com (static-content.springer.com)... 151.101.0.95, 151.101.64.95, 151.101.128.95, ...
Connecting to static-content.springer.com (static-content.springer.com)|151.101.0.95|:443... connected.
HTTP request sent, 

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
(data2.species)+'_'+(data2.Donor)

2678                s__Bacteroides_clarus_aa
5378          s__Bacteroides_intestinalis_aa
9158               s__Ruminococcus_bromii_aa
12938                  s__GGB6601_SGB9333_aa
13478                  s__GGB3256_SGB4303_aa
                         ...                
86343     s__Faecalibacterium_prausnitzii_dl
2103          s__Phocaeicola_massiliensis_dl
67983         s__Phocaeicola_massiliensis_dl
5883      s__Faecalibacterium_prausnitzii_dl
178143            s__Phocaeicola_plebeius_dl
Length: 208440, dtype: object

In [ ]:
data2[data2['value']>1]

,time,Donor,Age,Sex,BMI,species,value
39938,0154,aa,29,Male,24.1,s__Desulfovibrio_piger,1.00422
11318,0154,aa,29,Male,24.1,s__Odoribacter_splanchnicus,1.12785
77198,0154,aa,29,Male,24.1,s__Odoribacter_splanchnicus,1.12785
73418,0154,aa,29,Male,24.1,s__Faecalibacterium_prausnitzii,1.14483
183578,0154,aa,29,Male,24.1,s__GGB3304_SGB4367,1.24406
...,...,...,...,...,...,...,...
86343,0006,dl,32,Male,26.1,s__Faecalibacterium_prausnitzii,2.21002
2103,0006,dl,32,Male,26.1,s__Phocaeicola_massiliensis,3.84088
67983,0006,dl,32,Male,26.1,s__Phocaeicola_massiliensis,3.84088
5883,0006,dl,32,Male,26.1,s__Faecalibacterium_prausnitzii,4.37472


## UMAP by species via CPU


In [ ]:
data=pd.read_csv('/content/PRJNA544527_mpa4out.txt',sep='\t',skiprows=1,index_col=0)

g = graphistry.nodes(cudf.from_pandas(data.dropna()))

t=time()
g3=g.umap(dbscan=True,engine='umap_learn')
print('\n Total ', np.round(time() - t,1), 'seconds passed')

g3.plot()



 Total  24.8 seconds passed


## UMAP by species via GPU

In [ ]:
data=pd.read_csv('/content/PRJNA544527_mpa4out.txt',sep='\t',skiprows=1,index_col=0)

g = graphistry.nodes(cudf.from_pandas(data.dropna()))

t=time()
g3=g.umap(dbscan=True,engine='cuml')
print('\n Total ', np.round(time() - t,1), 'seconds passed')


 Total  0.7 seconds passed


In [ ]:
g3.plot()

## UMAP for patient by time stamp

In [ ]:
data2=data2[data2.value>0]
data2=data2.reset_index(drop = True)
data2=data2.drop_duplicates()

data2["Label"] = (
    data2.groupby("Donor")
    .apply(lambda x: x.groupby("time", sort=False).ngroup() + 1)
    .values
)

cc=pd.unique(data2[data2.Label<5].Donor)
data2=data2.loc[ data2.Donor.isin(cc), : ]
data2=data2[data2.Label<5]

data2["rank"] = data2.groupby("Donor")["value"].rank(method="dense", ascending=False)
data2=data2[data2['rank']<10.0]


In [ ]:
data2['id_time']=data2['Donor']+'_'+data2['Label'].apply(str)

In [ ]:
data3=data2[['id_time','species','value']]

In [ ]:
df2 = data3.pivot_table(index=['id_time'],columns='species')
df3=df2.fillna(0).reset_index()
df4=df3.droplevel(0, axis=1)
df4.index=df4.iloc[:,0]
df4=df4.loc[:, df4.columns.str.startswith('s__')]

g = graphistry.nodes(cudf.from_pandas(df4))

t=time()

g3=g.umap(dbscan=True,engine='cuml')
print('\n Total ', np.round(time() - t,1), 'seconds passed')

g3.plot()


 Total  0.3 seconds passed
